In [2]:
import os
import glob
import numpy as np

from PIL import Image
from urllib.request import urlopen

In [1]:
API_key = "insert API key here"

coords = [(14.780853203244734, 120.90955600885192), (14.416295592618148, 121.14383692812082), "Manila",
          (10.427664912970894, 123.77000234207509), (10.23705956509266, 124.01872390816466), "Cebu",
          (7.210947405536346, 125.34628743433869), (7.0255902591203245, 125.6628045194874), "Davao",
          (8.5248031891031, 124.5870501754955), (8.424525868259613, 124.6667109618823), "CDO",
          (10.856901539513862, 122.45210183587388), (10.683330220860539, 122.60222523796101), "Iloilo",
          (13.6544673064893, 123.1631603576015), (13.610242583616946, 123.2466802059456), "Naga"]

cities = []

areas = []
for i in range(0, len(coords), 3):
    name = coords[i + 2]
    topleft = coords[i]
    bottomright = coords[i + 1]
    h = bottomright[0] - topleft[0]
    w = bottomright[1] - topleft[1]
    area = h * w

    cities.append((topleft[0], topleft[1], h, w, name))
    areas.append(area)
    
probs = np.array(areas) / np.sum(areas)

outdir = "./out/"
os.makedirs(outdir, exist_ok=True)

# find existing indices
existing_files = glob.glob(os.path.join(outdir, "*.png"))
existing_indices = [int(os.path.basename(filename).split("_")[0]) for filename in existing_files]

num_images = 800
city_indices = np.random.choice(len(areas), size=(num_images), p=probs)
random_coords = np.random.rand(num_images, 2)

for i in range(0, num_images):
    if i in existing_indices:
        continue
    city_idx = city_indices[i]

    y, x, h, w, name = cities[city_idx]
    loc = (y + h * random_coords[i, 0], x + w * random_coords[i, 1])

    savename = "%06d_%s_%.5f_%.5f.png" % (i, name.replace(" ", "_"), loc[0], loc[1])
    
    styles = ["satellite", "roadmap"]
    
    out = Image.new('RGB', (1024, 512))
    for j, style in enumerate(styles):
        url = "https://maps.googleapis.com/maps/api/staticmap?center={},{}"+
        "&zoom=17&size=640x640&maptype={}&style=feature:all%7Celement:"+
        "labels%7Cvisibility:off&key={}".format(loc[0], loc[1], style, API_key)
        
        #url = sign_url(url, SECRET)
        im = Image.open(urlopen(url))
        im = im.crop((0, 0, 512, 512))
        out.paste(im, (512 * j, 0))
    out.save(os.path.join(outdir, savename))
    
    if i % 10 == 0:
        print(savename)